In [1]:
import tensorflow as tf
import gym 
import numpy as np

In [2]:
def discount(reward, gamma):
    discount = np.zeros_like(reward)
    G = 0.0
    for i in reversed(range(0, len(reward))):
        G = G * gamma + reward[i]
        discount[i] = G

    mean = np.mean(discount)
    std = np.std(discount)
    discount = (discount - mean) / (std)
    return discount

In [3]:
num_inputs = 4
num_hidden = 4
num_outputs = 2
learning_rate = 0.01
gamma = 0.9

In [62]:
# Define the model

X = tf.placeholder(tf.float32,shape=(None, num_inputs))
Y = tf.placeholder(tf.int32,shape=(None,2))
reward = tf.placeholder(tf.float32)

hidden_one = tf.layers.dense(X,num_hidden,activation=tf.nn.relu)
#hidden_two = tf.layers.dense(hidden_one,num_hidden,activation=tf.nn.relu)
logits = tf.layers.dense(hidden_one, num_outputs,activation=None)

probabilities = tf.nn.softmax(logits)
expected_reward = tf.layers.dense(logits,1, activation=tf.nn.tanh)
log_prob = tf.log(probabilities)

action = tf.multinomial(probabilities, num_samples=1)

In [63]:
cross_entropy = tf.losses.softmax_cross_entropy(Y, logits)
#mse = tf.losses.mean_squared_error(reward,expected_reward)
err_sum = cross_entropy 

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(err_sum)

In [64]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [65]:
def rollout(env,sess, max_steps, gamma):
    """
    Plays a single game
    Returns: Input, discounted_reward, action_taken
    """
    obs = env.reset()
    inputs = []
    rewards = []
    actions = []
    done = False
    
    for step in range(max_steps):
        inputs.append(obs.reshape(1,4))
        act = sess.run(action, feed_dict={X:obs.reshape(1,4)})
        act = act[0][0]
        
        actions.append(act)
        obs, reward, done, _ = env.step(act)
    
        rewards.append(reward)
        if done:
            break
    discounted_rewards = discount(rewards, gamma)
    return inputs, discounted_rewards, actions
            

In [66]:
def num_to_one_hot(number):
    vector = np.zeros((1,2))
    vector[0][number] = 1
    return vector
def reverse_to_one_hot(number):
    vector = np.zeros((1,2))
    vector[0][np.abs(1-number)] = 1
    return vector

In [67]:
num_to_one_hot(1)


array([[0., 1.]])

In [ ]:
epoch_num = 20000
threshold = 50

with tf.Session() as sess:
    sess.run(init)
    env = gym.make('CartPole-v1')
    
    
    for episode in range(epoch_num):
        observations, rewards, actions = rollout(env,sess,500,0.9)
        lengths = []
        
        
        one_hot_actions = np.array([num_to_one_hot(a) for a in actions]).reshape(-1,2)
        feed_dict = {X:np.array(observations).reshape(-1,4)*rewards.reshape(-1,1), Y:one_hot_actions, reward:rewards}
        sess.run(optimizer,feed_dict=feed_dict)
#         else:
#             one_hot_actions = np.array([reverse_to_one_hot(a) for a in actions]).reshape(-1,2)

#         feed_dict = {X:np.array(observations).reshape(-1,4), Y:one_hot_actions, reward:rewards}


#         sess.run(optimizer,feed_dict=feed_dict)
        #print(len(rewards), end=', ')
        lengths.append(len(rewards))
        if episode % 100 == 0:
            avg = np.mean(lengths)
            lengths = []
            print('Avg run: {}'.format(avg))
#             print(sess.run(tf.trainable_variables()[0]))
       
   

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Avg run: 10.0
Avg run: 20.0
Avg run: 22.0
Avg run: 15.0
Avg run: 45.0
Avg run: 25.0
Avg run: 17.0
Avg run: 36.0
Avg run: 17.0
Avg run: 40.0
Avg run: 40.0
Avg run: 21.0
Avg run: 13.0
Avg run: 38.0
Avg run: 20.0
Avg run: 63.0
Avg run: 23.0
Avg run: 10.0
Avg run: 31.0
Avg run: 14.0
Avg run: 11.0
Avg run: 28.0
Avg run: 17.0
Avg run: 62.0
Avg run: 21.0
Avg run: 45.0
Avg run: 33.0
Avg run: 11.0
Avg run: 17.0
Avg run: 52.0
Avg run: 42.0
Avg run: 31.0
Avg run: 20.0
Avg run: 21.0
